In [1]:
import pandas as pd
#import json
from datetime import date

# Collecting the data from Twitter

In [2]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [3]:
import pysad
import pysad.utils
import pysad.collect

In [4]:
import importlib
importlib.reload(pysad.collect)

<module 'pysad.collect' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad/collect.py'>

In [5]:
init_accounts = pysad.utils.initial_accounts()

In [6]:
init_accounts.list()

['swiss_climate_regular',
 'swiss_climate_controversial',
 'swiss_immigration',
 'french_tech_lesechos',
 'swiss_immigration2',
 'debat_burqa',
 'coronavirus',
 'coronavirus_iran',
 'hackathlon',
 'hackathlon_popular',
 'hackathlon_reddit',
 'hackathlon_missingtweets']

In [7]:
######Choose a category##############    
#category_name = 'swiss_climate_controversial'
#category_name = 'swiss_climate_regular'
#category_name = 'french_tech_lesechos'
#category_name = 'swiss_immigration'
#category_name = 'swiss_immigration2'
#category_name = 'debat_burqa'
category_name = 'hackathlon'
category_name = 'hackathlon_popular'
category_name = 'hackathlon_reddit'
category_name = 'hackathlon_missingtweets'

#####################################

username_list = init_accounts.accounts(category_name)

# create the path to save the experiment indexed with the date of today
today = date.today()
date_string = today.strftime("%Y%m%d")
print("date string =", date_string)

tweet_data_path_list = ['tweetdata', category_name, date_string]
results_data_path_list = ['resultsdata2', category_name, date_string]


date string = 20200411


In [9]:
tweet_data_path = ''.join(tweet_data_path_list)
results_data_path = ''.join(results_data_path_list)

In [13]:
# Initialize folders (create or clean them if they exist)
# Set erase=False if you need to keep the previous collection
tweet_data_path = pysad.utils.initialize_folder(tweet_data_path_list, erase=False)
results_data_path = pysad.utils.initialize_folder(results_data_path_list, erase=False)

In [ ]:
# If the list of users is too large:
# Keep a random subset of the list of users
import random
username_list_tmp = random.sample(username_list,len(username_list)//10)
username_list = username_list_tmp

In [ ]:
len(username_list)

In [ ]:
username_list

In [ ]:
min_mentions = 2 # minimal number of mentions of a user to be followed
max_day_old = None # number max of days in the past
exploration_depth = 1 # mention of mention of mention of ... up to exploration depth

In [ ]:
total_user_list = pysad.collect.collect_tweets(username_list, tweet_data_path, python_tweets, min_mentions=min_mentions,
               max_day_old=max_day_old, exploration_depth=exploration_depth)

In [ ]:
len(total_user_list)

# Loading the saved data into an edge table

In [14]:
import glob

edge_df = pd.DataFrame()
#for filename in glob.glob(tweet_data_path + '*_mentions' +'_t' +str(min_mentions)+ '.json'):
for filename in glob.glob(tweet_data_path + '*_mentions*' + '.json'):
    new_edge_df = pd.read_json(filename)
    print('{} with {} tweets.'.format(filename,len(new_edge_df)))
    edge_df = edge_df.append(new_edge_df)
edge_df.reset_index(drop=True, inplace=True)

tweetdata/hackathlon_missingtweets/20200411/Alicia_Smith19_mentions.json with 41 tweets.
tweetdata/hackathlon_missingtweets/20200411/nygovcuomo_mentions.json with 32 tweets.
tweetdata/hackathlon_missingtweets/20200411/blue500000_mentions.json with 1 tweets.
tweetdata/hackathlon_missingtweets/20200411/nytimes_mentions.json with 25 tweets.
tweetdata/hackathlon_missingtweets/20200411/AskAKorean_mentions.json with 64 tweets.
tweetdata/hackathlon_missingtweets/20200411/BFMTV_mentions.json with 24 tweets.
tweetdata/hackathlon_missingtweets/20200411/m_gramatyka_mentions.json with 39 tweets.
tweetdata/hackathlon_missingtweets/20200411/HadiNili_mentions.json with 75 tweets.
tweetdata/hackathlon_missingtweets/20200411/jasminjoestar_mentions.json with 89 tweets.
tweetdata/hackathlon_missingtweets/20200411/Freaktesam_mentions.json with 64 tweets.
tweetdata/hackathlon_missingtweets/20200411/_pbls_mentions.json with 170 tweets.
tweetdata/hackathlon_missingtweets/20200411/itrulyknowchina_mentions.jso

In [ ]:
# Keeping only the most popular
df_pop = edge_df[edge_df['retweet_count'] > 1000]
df_pop

## Creating the graph

In [16]:
import pysad.graph
import pysad.clusters

In [17]:
import importlib
importlib.reload(pysad.graph)

<module 'pysad.graph' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad/graph.py'>

In [18]:
DEGREE_MIN = 2 # Minimal number of connections in the graph

G = pysad.graph.graph_from_edgeslist(edge_df,DEGREE_MIN)
#G = pysad.graph.graph_from_edgeslist(df_pop,DEGREE_MIN)
G.name = category_name
G.end_date = max(edge_df['date']) #max(edge_df['date'].apply(max))
G.start_date = min(edge_df['date']) #min(edge_df['date'].apply(min))
print('Period from {} to {}.'.format(G.start_date,G.end_date))

Creating the graph from the edge list
Nb of nodes: 2769
Nb of nodes after removing nodes with degree strictly smaller than 2: 186
removed 5 isolated nodes.
Period from 2016-07-27 12:40:55 to 2020-04-11 13:36:55.


## Community detection to get the clusters

In [19]:
import importlib
importlib.reload(pysad.clusters)
importlib.reload(pysad.graph)

<module 'pysad.graph' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad/graph.py'>

In [20]:
G,clusters = pysad.graph.detect_communities(G)
G.nb_communities = len(clusters)
c_connectivity = pysad.clusters.cluster_connectivity(G)

Communities saved on the graph as node attributes.
Nb of partitions: 10


In [21]:
G = pysad.graph.remove_small_communities(G,clusters,min_size=5)

removed 0 community(ies) smaller than 5 nodes.


In [ ]:
# Save the graph
import networkx as nx

graphname = 'missinggraph_reddit'
graphfilename = results_data_path + graphname + '_t' + str(min_mentions) + '_md' + str(DEGREE_MIN) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
print('Wrote',graphfilename)

In [22]:
c_connectivity

{0: 0.344,
 1: 0.17,
 2: 0.341,
 3: 0.61,
 4: 0.338,
 5: 0.35,
 6: 0.139,
 7: 0.333,
 8: 0.359,
 9: 0.295}

## Analyze clusters

In [23]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad/__init__.py'>

## Automatic processing of all clusters

In [25]:
# Extracting the data from the clusters
cluster_info_dic = {}
for c_id in clusters:
    cgraph = clusters[c_id]
    cgraph = pysad.clusters.cluster_attributes(cgraph)
    table_dic = pysad.clusters.cluster_tables(cgraph)
    cluster_filename = results_data_path + 'cluster' + str(c_id)
    cluster_info_dic[c_id] = {}
    cluster_info_dic[c_id]['info_table'] = table_dic
    cluster_info_dic[c_id]['filename'] = cluster_filename    

In [26]:
# Adding global infos
# keywords
corpus = pysad.clusters.get_corpus(cluster_info_dic)
keyword_dic = pysad.clusters.tfidf(corpus)

In [29]:
# gathering global info
# Saving in excel files
for c_id in cluster_info_dic:
    info_table = cluster_info_dic[c_id]['info_table']
    info_table['keywords'] = keyword_dic[c_id]
    cluster_general_info = {'cluster id': c_id, 'Nb users': clusters[c_id].number_of_nodes(), 
                           'Nb of tweets':clusters[c_id].size(weight='weight'),
                           'Start date': str(G.start_date),
                           'End date': str(G.end_date),
                           'Search topic': category_name,
                           'cluster connectivity': c_connectivity[c_id]}
    cluster_general_df = pd.DataFrame.from_dict([cluster_general_info])
    #info_table = {'cluster':cluster_general_df, **info_table}
    sheet1 = pd.concat([cluster_general_df,info_table['hashtags'],info_table['keywords']],axis=1)
    tweet_table = info_table['text']
    cluster_indicators = pd.DataFrame([pysad.clusters.compute_cluster_indicators(clusters[c_id])])
    excel_data = {'cluster':sheet1, 'tweets':tweet_table, 'indicators': cluster_indicators}
    #excel_data = info_table
    pysad.clusters.save_excel(excel_data,cluster_info_dic[c_id]['filename'] + '_infos.xlsx', table_format='Fanny')
    pysad.graph.save_graph(clusters[c_id],cluster_info_dic[c_id]['filename'] + 'graph.gexf')

Data saved to resultsdata2/hackathlon_missingtweets/20200411/cluster0_infos.xlsx
Graph saved to resultsdata2/hackathlon_missingtweets/20200411/cluster0graph.gexf
Data saved to resultsdata2/hackathlon_missingtweets/20200411/cluster1_infos.xlsx
Graph saved to resultsdata2/hackathlon_missingtweets/20200411/cluster1graph.gexf
Data saved to resultsdata2/hackathlon_missingtweets/20200411/cluster2_infos.xlsx
Graph saved to resultsdata2/hackathlon_missingtweets/20200411/cluster2graph.gexf
Data saved to resultsdata2/hackathlon_missingtweets/20200411/cluster3_infos.xlsx
Graph saved to resultsdata2/hackathlon_missingtweets/20200411/cluster3graph.gexf
Data saved to resultsdata2/hackathlon_missingtweets/20200411/cluster4_infos.xlsx
Graph saved to resultsdata2/hackathlon_missingtweets/20200411/cluster4graph.gexf
Data saved to resultsdata2/hackathlon_missingtweets/20200411/cluster5_infos.xlsx
Graph saved to resultsdata2/hackathlon_missingtweets/20200411/cluster5graph.gexf
Data saved to resultsdata2/h